## Actualizando la información de los LLMs

Una gran limitación de los  LLMS es que almacenan toda la información en sus parámetros que son ajustados en el momento de entrenamiento. Dentro de la memoria de ChatGPT hay almacenada una gran cantidad de datos de la realidad que:
* Se repitieron en los textos de entrenamiento de la etapa no supervisada (predicción del texto que sigue)

* Fueron parte de las respuestas escritas por humanos en la etapa supervisada

* Fueron etiquetadas por un humano como correctas (:thumbsup:) en la etapa de RLHF

Miren el siguiente ejemplo:




<img src="https://i.ibb.co/wp3vrv5/Screen-Shot-2023-09-17-at-17-03-57.png" width="500px" />




Pero ¿Qué pasa si necesitamos datos recientes?

<img src="https://i.ibb.co/rMj3ScT/Whats-App-Image-2023-09-13-at-13-36-03-1.jpg" width="500px" />



```
# Tiene formato de código
```

Desde la aparición de los LLMs cobraron gran importancia las herramientas capaces de proveer información textual en tiempo real, como las bases de datos vectoriales (que vamos a ver más adelante en el curso) y los motores de búsqueda consumidos a través de APIs como Bing y <a href="https://serpapi.com/">SERP API</a>, que utiliza web scraping de los resultados orgánicos de Google.

Existen aplicaciones para usuarios finales que ya permiten combinar la capacidad de razonamiento de los LLMs con estos grandes índices de información actualizada como https://www.bing.com/chat y https://www.perplexity.ai/

<img src="https://i.ibb.co/VQ27VQZ/Whats-App-Image-2023-09-13-at-13-36-03.jpg" width="500px" />


## Ejercicio

Les proponemos utilizar las APIs de SERP y OpenAI para responder preguntas de actualidad, como por ejemplo ¿Quién ganó las elecciones de Argentina 2023? Exploremos un sistema capaz de responder a cualquier pregunta de actualidad.

Para poder resolver este ejercicio vamos a utilizar dos sistemas externos: SERP API y OpenAI. Ambos tienen consumos de prueba que se pueden utilizar para explorar, pero cuando integramos este tipo de servicios a nuestras tenemos que considerar:

1) Los costos: https://serpapi.com/pricing // https://openai.com/pricing

2) Seguridad: ¡Cuidado! Las claves privadas son lo único que se necesita para acceder con una identidad. En los sistemas productivos las claves se encuentran en un archivo de configuración (.env) y JAMÁS se deben subir a los repositorios de código como git. Normalmente se excluye al .env del repositorio utilizando el archivo .gitignore


In [ ]:
# Instalamos las librerías
!pip install google-search-results openai dotenv

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=66158e03f34376fa8dc95ef858e189865cd881813f4e7ea471306d68954a7126
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [ ]:
# Paso 1. Registrarse a SERP API y Open AI para obetener las siguientes claves
from dotenv import load_dotenv
import os
load_dotenv()

SERP_API_KEY = os.getenv("SERP_API_KEY")
#OPENAI_ORGANIZATION = userdata.get("OPENAI_ORGANIZATION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# Paso 2.Siguiendo la documentación de SERP API escriban una función para buscar en SERP. Por default vamos a querer 5 resultados

from itertools import islice
from dataclasses import dataclass
from serpapi import GoogleSearch

@dataclass(frozen=True)
class SnippetCitation:
    Url: str
    Title: str
    Snippet: str



def serp_results(query: str, num=5, api_key=SERP_API_KEY):
  params = ? # Busquen los parámetros en la documentación de SERP
  search = GoogleSearch(params)
  res = search.get_dict()
  organic_results = res["organic_results"]
  return_results = []
  for result in organic_results:
      try:
        return_results.append(SnippetCitation(Url=result["link"], Title=result["title"], Snippet=result["snippet"]))
      except Exception as e:
          print(e)
  return return_results

SyntaxError: invalid syntax (ipython-input-4280338802.py, line 16)

In [6]:
serp_res = serp_results("¿Quién gano las elecciones presidenciales en Croacia en 2025?")
serp_res

[]

# Paso 3

Utilizar GPT para responder a la pregunta.

Siguiendo la documentación de la API de OpenAI, escriban una función para generar texto simple


In [ ]:
from openai import OpenAI
client = # ¿Cómo crear el cliente utilizando la API key?

In [ ]:
  # Uses OPENAI_API_KEY environment variable

def chat_complete(
    syst: str | None,
    user: list[str] = [],
    assistant: list[str] = [],
    max_tokens: int = 1024,
    temperature: float = 0,
    model: str = ?, # Elijan el modelo por default. ¿Cuánto cuesta por cada 1000 tokens de output? ¿Y de input? https://platform.openai.com/docs/models
) -> str:
    # Initialize the OpenAI client
    messages: list[dict[str, str]] = []

    if syst is not None:
        messages.append({"role": "system", "content": syst})

    for i in range(len(user)):
        messages.append({"role": "user", "content": user[i]})
        if len(assistant) > i:
            messages.append({"role": "assistant", "content": assistant[i]})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
    )

    return response.choices[0].message.content

In [ ]:
chat_complete("Hola")

# Paso 4

Pensemos un buen prompt que pueda tomar como contexto la información obtenida de SERP API

In [ ]:

SYSTEM_PROMPT = """Tu objetivo es responder a la pregunta [PREGUNTA] utilizando este contexto [CONTEXTO].

Si la respuesta no se encuentra en el contexto, responde que no sabes. """


In [ ]:
pregunta = "¿Quién gano las elecciones presidenciales en Croacia en 2025?"

In [ ]:
SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[PREGUNTA]", pregunta)
SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[CONTEXTO]", str(serp_res))

In [ ]:
chat_complete(SYSTEM_PROMPT)

# Paso 6

Darle una forma estructurada a las respuestas.

Ahora vamos a adaptar la función chat_complete para recibir como parámetro el schema


In [ ]:
def limpiar_markdown(content: str):
    # Remove markdown code block markers
    content = content.strip()
    if content.startswith('```json'):
        content = content[7:]  # Remove ```json
    elif content.startswith('```'):
        content = content[3:]   # Remove ```
    if content.endswith('```'):
        content = content[:-3]  # Remove closing ```
    content = content.strip()
    return content

In [ ]:
def chat_complete(
    syst: str | None,
    user: list[str] = [],
    assistant: list[str] = [],
    max_tokens: int = 1024,
    temperature: float = 0,
    model: str = "gpt-4o",
    schema: dict | None = None
) -> str:
    # Initialize the OpenAI client
    messages: list[dict[str, str]] = []

    if syst is not None:
        messages.append({"role": "system", "content": syst})

    for i in range(len(user)):
        messages.append({"role": "user", "content": user[i]})
        if len(assistant) > i:
            messages.append({"role": "assistant", "content": assistant[i]})

    # Build the request parameters
    request_params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
    }
    if schema is not None:
        request_params["response_format"] = {
            "type": "json_schema",
            "json_schema": {
                "name": "structured_response",
                "schema": schema,
                "strict": True
            }
        }
    response = client.chat.completions.create(**request_params)
    content = response.choices[0].message.content
    content = limpiar_markdown(content)
    return content


In [ ]:
SYSTEM_PROMPT = """Tu objetivo es responder a la pregunta [PREGUNTA] utilizando este contexto [CONTEXTO].

El formato de salida debe ser un array de json con todas las entidades mencionadas. Cada una representada como un único string del título """

In [ ]:
pregunta = "Recomendaciones de series 2025"

In [ ]:
serp_res = serp_results(pregunta)

In [ ]:
SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[PREGUNTA]", pregunta)
SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[CONTEXTO]", str(serp_res))

In [ ]:
# Ahora representemos en un diccionario un esquema con un array de string
RECOMMENDATIONS_SCHEMA = {
    "type": "ARRAY",
    "items": {
        "type": "STRING"
    }
}

In [ ]:
json_list = chat_complete(SYSTEM_PROMPT)

In [ ]:
import json

json.loads(json_list)

# Paso 7. Bonus

Como extra, hagamos una función que nos indique dónde ver cada una de las series.

In [ ]:
shows = json.loads(json_list)

In [ ]:
def get_streaming_service(show:str):
  SYSTEM_PROMPT = """Tu objetivo es responder a la pregunta [PREGUNTA] utilizando este contexto [CONTEXTO]."""
  pregunta = f"¿Dónde puedo ver la serie {show}?"
  contexto = serp_results(pregunta)
  SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[PREGUNTA]", pregunta)
  SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[CONTEXTO]", str(contexto))
  streaming = chat_complete(SYSTEM_PROMPT)
  print(show,"-->" ,streaming)
  return (show, streaming)

In [ ]:
results = []
# Hagamos sólo 5 para ahorrar costos
for show in shows[:5]:
  results.append(get_streaming_service(show))